In [1]:
from tasho.templates.P2P import P2P
from tasho import TaskModel, default_mpc_options
from tasho.templates.Regularization import Regularization
from tasho.Variable import Variable
from tasho.ConstraintExpression import ConstraintExpression
from tasho.Expression import Expression
from tasho.MPC import MPC 
from tasho import environment as env
from tasho import WorldSimulator
from tasho.OCPGenerator import OCPGenerator
from robotshyu import Robot as rob
from time import sleep
import numpy as np
import casadi as cs
import pybullet as p
import tasho
import robotshyu
import os

pybullet build time: May 20 2022 19:44:17


In [2]:
# Options for example codes
frame_enable = True

# Ex1) P2P example

In [3]:
robot = rob.Robot("indy7")
link_name = 6
goal_pose = Variable(robot.name, "goal_pose", "magic_number", (4,4), np.array(
        [[0, 0, 1, 1], [0, 1, 0, -0.1], [-1, 0, 0, 0.6], [0, 0, 0, 1]]
    ))
q0 = [1.0, -0.05311582280659044, -0.815452580946695, -1.3191046402052224, -0.8582660722530533, 1.3988994390898029]

robot.set_joint_acceleration_limits(lb=-360*3.14159/180, ub=360*3.14159/180)
ndof = robot.nq
q_current = Variable(robot.name, "jointpos_init", 'magic_number', (ndof, 1), q0)

Loading robot params from json ...
Loaded 6-DoF robot: indy7


In [4]:
task_P2P = P2P(robot, link_name, goal_pose, q_current, 0.001)

In [5]:
task_P2P.write_task_graph("Indy7_P2P_Task.svg")

In [6]:
task_P2P.add_path_constraints(Regularization(task_P2P.variables['qd_'+robot.name], 1),
                              Regularization(task_P2P.variables['qdd_'+robot.name], 0.1))

In [7]:
task_P2P.write_task_graph("Indy7_P2P_Task2.svg")

In [8]:
horizon_steps = 10
horizon_period = 3.0
q_pred = [0]*horizon_steps

OCP_gen = OCPGenerator(task_P2P, False, {"time_period": horizon_period, "horizon_steps":horizon_steps})
q_ocp = OCP_gen.stage_tasks[0].variables['q_'+robot.name].x
OCP_gen.tc.set_initial(q_ocp, q0)
OCP_gen.tc.set_ocp_solver(
    "ipopt", 
    # {"ipopt":{"linear_solver":"ma27"}}
)
OCP_gen.tc.solve_ocp()


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:      452
Number of nonzeros in inequality constraint Jacobian.:      186
Number of nonzeros in Lagrangian Hessian.............:      141

Total number of variables............................:      192
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      141
Total number of inequality c

In [9]:
t_grid, qsol = OCP_gen.tc.sol_sample(q_ocp)
_, q_dot_sol = OCP_gen.tc.sol_sample(OCP_gen.stage_tasks[0].variables['qd_'+robot.name].x)

obj = WorldSimulator.WorldSimulator()

# Add robot to the world environment
position = [0.0, 0.0, 0.0]
orientation = [0.0, 0.0, 0.0, 1.0]

robotID = obj.add_robot(position,orientation,robot_pkg=robot)
frameID0 = obj.add_robot(position,orientation,robot_pkg=robot,frame=True)
robotIDs=[robotID,frameID0]

if frame_enable:
    frameIDs = [0]*horizon_steps
    for i in range(horizon_steps):
        frameIDs[i] = obj.add_robot(position,orientation,robot_pkg=robot,frame=True)


for i in range(horizon_steps):
    q_pred[i]=qsol[i]

joint_indices = [0, 1, 2, 3, 4, 5]

obj.resetMultiJointState(robotIDs, joint_indices, [q0])

if frame_enable:
    obj.resetMultiJointState(frameIDs, joint_indices, [q0])
    obj.resetMultiJointState(frameIDs,joint_indices,q_pred)

In [10]:
for i in range(horizon_steps + 1):
    sleep(horizon_period*0.5/horizon_steps)
    obj.resetMultiJointState(
        robotIDs, joint_indices, [qsol[i]]
    )
    
sleep(0.5)
obj.end_simulation()

Ending simulation


# Ex2) Object picking MPC

In [11]:
robot = rob.Robot("indy7")
link_name = 6

# Define initial conditions of the robot
q0_val = [0, -0.423598, -0.9, 0.0, -1.5709, -0.523598]
qd0_val = [0] * robot.ndof

robot.set_joint_acceleration_limits(lb=-360*3.14159/180, ub=360*3.14159/180)

################################################
# Task spacification - Approximation to object
################################################

# Select prediction horizon and sample time for the MPC execution
horizon_size = 10
t_mpc = 0.05

q_init = Variable(robot.name, 'q_init', 'parameter', (robot.ndof,1))
qd_init = Variable(robot.name, 'qd_init', 'parameter', (robot.ndof,1))
goal_pose = Variable(robot.name, "goal_pose", 'parameter', (4,4))

task_P2P = P2P(robot, link_name, goal_pose, q_init, 0.001)
task_P2P.write_task_graph("Indy7_P2P_Task4.svg")

Loading robot params from json ...
Loaded 6-DoF robot: indy7


In [12]:
task_P2P.remove_initial_constraints(task_P2P.constraint_expressions['stationary_qd_indy7'])
task_P2P.remove_terminal_constraints('rot_con_pose_6_indy7_vs_goal',
                                    'trans_con_pose_6_indy7_vs_goal')
task_P2P.write_task_graph("Indy7_P2P_Task5.svg")

In [13]:
task_P2P.add_initial_constraints(ConstraintExpression(robot.name, "eq", Expression(robot.name, "err_qd_qinit", lambda a, b : a - b, qd_init, task_P2P.variables['qd_indy7']),
                             "hard", reference = 0))
task_P2P.add_path_constraints(Regularization(task_P2P.expressions['trans_error_pose_6_indy7_vs_goal'], 5e0, norm = "L1"),
                            Regularization(task_P2P.expressions['ax_ang_error_pose_6_indy7_vs_goal'], 1e0), 
                            Regularization(task_P2P.variables['qd_indy7'], 1e-1),
                            Regularization(task_P2P.variables['qdd_indy7'], 1e-3))

task_P2P.write_task_graph("Indy7_P2P_Task6.svg")

In [14]:
pOCP = OCPGenerator(task_P2P, False, {"time_period":horizon_size*t_mpc, "horizon_steps":horizon_size})

tc = pOCP.tc

################################################
# Set solver and discretization options
################################################
tc.set_ocp_solver("ipopt", {"ipopt": {"print_level": 5,"tol": 1e-3}})
# tc.set_ocp_solver("ipopt", {"ipopt": {"print_level": 0,"tol": 1e-3, "linear_solver":"ma27"}}) #use this if you have hsl


################################################
# Set parameter values
################################################
q = pOCP.stage_tasks[0].variables['q_indy7'].x
qd = pOCP.stage_tasks[0].variables['qd_indy7'].x
q_0 = pOCP.stage_tasks[0].variables['q_init_indy7'].x
qd_0 = pOCP.stage_tasks[0].variables['qd_init_indy7'].x
goal_pose = pOCP.stage_tasks[0].variables['goal_pose_indy7'].x
goal_pose_val =  cs.vertcat(
    cs.hcat([-1, 0, 0, 0.5]),
    cs.hcat([0, 1, 0, 0.0]),
    cs.hcat([0, 0, -1, 0.25]),
    cs.hcat([0, 0, 0, 1]),
)
tc.set_initial(q, q0_val)
tc.set_value(goal_pose, goal_pose_val)
tc.set_value(q_0, q0_val)
tc.set_value(qd_0, qd0_val)

# Add an output port for joint velocities as well
tc.tc_dict["out_ports"].append({"name":"port_out_qd_indy7", "var":"qd_indy7", "desc": "output port for the joint velocities"})

# Add a monitor for termination criteria
tc.add_monitor({"name":"termination_criteria", "expression":cs.sqrt(cs.sumsqr(pOCP.stage_tasks[0].expressions["trans_error_pose_6_indy7_vs_goal"].x)) - 2e-2, "reference":0, "lower":True, "initial":True})

mpc_options = default_mpc_options.get_default_mpc_options()


tc.ocp_solver = "ipopt"
tc.ocp_options = mpc_options["ipopt_hsl"]["options"]
tc.mpc_solver = tc.ocp_solver
tc.mpc_options = tc.ocp_options
tc.set_ocp_solver("ipopt", tc.mpc_options)
os.system("export OMP_NUM_THREADS = 1")

sol = tc.solve_ocp()

      solver startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa/X.org
GL_RENDERER=llvmpipe (LLVM 12.0.0, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 21.2.6
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 21.2.6
Vendor = Mesa/X.org
Renderer = llvmpipe (LLVM 12.0.0, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = Mesa/X.org
ven = Mesa/X.org
numActiveThreads = 0
stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main sema

sh: 1: export: : bad variable name


RuntimeError: Error in Opti::solve [OptiNode] at .../casadi/core/optistack.cpp:157:
.../casadi/core/optistack_internal.cpp:998: Assertion "return_success(accept_limit)" failed:
Solver failed. You may use opti.debug.value to investigate the latest values of variables. return_status is 'Invalid_Option'

In [15]:
dir_casadi_func = "casadi_dir"
os.makedirs("./"+dir_casadi_func, exist_ok=True)
vars_db = tc.generate_MPC_component("./"+dir_casadi_func+"/", {"ocp_cg_opts":{"save":True, "codegen":False, "jit":False}, "mpc":True, "mpc_cg_opts":{"save":True, "codegen":False, "jit":False}})

@1=(opti1_p_2[12:15]-fk_T(opti1_x_1){6}[12:15]), (sqrt(dot(@1, @1))-0.02)
@1=(opti1_p_2[12:15]-fk_T(opti1_x_1){6}[12:15]), (sqrt(dot(@1, @1))-0.02)


In [8]:
MPC_component = MPC("indy7_obj_pickup", "./"+dir_casadi_func+"/"+ tc.name + ".json")

In [9]:
obj = WorldSimulator.WorldSimulator(bullet_gui=True)

# Add robot to the world environment
position = [0.0, 0.0, 0.0]
orientation = [0.0, 0.0, 0.0, 1.0]

robotID = obj.add_robot(position,orientation,robot_pkg=robot)

if frame_enable:
    frameIDs = [0]*horizon_size
    for i in range(horizon_size):
        frameIDs[i] = obj.add_robot(position,orientation,robot_pkg=robot,frame=True)

# Set environment
environment = env.Environment()
package_path = tasho.__path__[0]
cube1 = env.Cube(length = 1, position = [0.5, -0.2, 0.35], orientation = [0.0, 0.0, 0.0, 1.0], urdf = package_path+"/models/objects/cube_small.urdf")
environment.add_object(cube1, "cube")
table1 = env.Box(height = 0.3, position = [0.5, 0, 0], orientation = [0.0, 0.0, 0.7071080798594737, 0.7071054825112364], urdf =package_path+ "/models/objects/table.urdf")
environment.add_object(table1, "table1")

cube2 = env.Cube(length = 1, position = [0.5, -0.2, 0.35], orientation = [0.0, 0.0, 0.0, 1.0], urdf = package_path+"/models/objects/cube_small_col.urdf", fixed = True)
environment.add_object(cube2, "cube2")
environment.set_in_world_simulator(obj)
cubeID = environment.get_object_ID("cube")
cube2ID = environment.get_object_ID("cube2")

p.resetBaseVelocity(cubeID, linearVelocity=[0, 0.8, 0])
# Determine number of samples that the simulation should be executed
no_samples = int(t_mpc / obj.physics_ts)
if no_samples != t_mpc / obj.physics_ts:
    print("[ERROR] MPC sampling time not integer multiple of physics sampling time")

# Correspondence between joint numbers in bullet and OCP
joint_indices = [0, 1, 2, 3, 4, 5]

 # Begin the visualization by applying the initial control signal
ts, q_sol = tc.sol_sample(q, grid="control")
ts, q_dot_sol = tc.sol_sample(qd, grid="control")

obj.resetJointState(robotID,joint_indices,q0_val)
if frame_enable==True:
    obj.resetMultiJointState(frameIDs, joint_indices, [q0_val])

obj.setController(
    robotID, "velocity", joint_indices, targetVelocities=q_dot_sol[0]
)
q_log = []
q_dot_log = []
predicted_pos_log = []
q_pred = [0]*horizon_size


In [11]:
# Execute the MPC loop

for i in range(horizon_size * 30):
    print("----------- MPC execution -----------")

    q_now = obj.readJointPositions(robotID, joint_indices)
    qd_now = obj.readJointVelocities(robotID, joint_indices)
        
    MPC_component.input_ports["port_inp_q_init_indy7"]["val"] = q_now
    MPC_component.input_ports["port_inp_qd_init_indy7"]["val"] = qd_now

    # Predict the position of the target object (cube)
    lin_vel, ang_vel = p.getBaseVelocity(cubeID)
    lin_vel = cs.DM(lin_vel)
    lin_pos, _ = p.getBasePositionAndOrientation(cubeID)
    lin_pos = cs.DM(lin_pos)
    time_to_stop = cs.norm_1(lin_vel) / 0.3
    predicted_pos = (
        cs.DM(lin_pos)
        + cs.DM(lin_vel) * time_to_stop
        - 0.5 * 0.5 * lin_vel / (cs.norm_1(lin_vel) + 1e-3) * time_to_stop ** 2
    )
    predicted_pos_log.append(predicted_pos.full())
    p.resetBasePositionAndOrientation(cube2ID, predicted_pos.full(), [0.0, 0.0, 0.0, 1.0])
    predicted_pos[2] += 0.06  # cube height
    print("Predicted position of cube", predicted_pos)
    predicted_pos_val = cs.vertcat(
        cs.hcat([-1, 0, 0, predicted_pos[0]]),
        cs.hcat([0, 1, 0, predicted_pos[1]]),
        cs.hcat([0, 0, -1, predicted_pos[2]]),
        cs.hcat([0, 0, 0, 1]),
    )

    MPC_component.input_ports["port_inp_goal_pose_indy7"]["val"] = cs.vec(predicted_pos_val)
        
    if i == 0:
        MPC_component.configMPC()

    MPC_component.runMPC()
    if frame_enable==True:
        sol = MPC_component.res_vals
        for i in range(horizon_size):
            q_pred[i]=sol[robot.ndof*i:robot.ndof*i+robot.ndof].full()
        obj.resetMultiJointState(frameIDs, joint_indices, q_pred)

    q_log.append(q_now)
    q_dot_log.append(qd_now)

    # Set control signal to the simulated robot
    qd_control_sig = MPC_component.output_ports["port_out_qd_indy7"]["val"].full()
    qdd_control_sig = (MPC_component.output_ports["port_out_qdd_indy7"]["val"] * t_mpc).full()
    
    obj.setController(
        robotID, "velocity", joint_indices, targetVelocities=qd_control_sig+qdd_control_sig
    )
    
    # Simulate
    obj.run_simulation(no_samples)

    # Termination criteria
#     if "termination_criteria_true" in MPC_component.event_output_port:
#         break

----------- MPC execution -----------
Predicted position of cube [0.5, 0.15777500346789, 0.41]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  86.00us ( 28.67us)  87.13us ( 29.04us)         3
       nlp_g  |  28.00us (  9.33us)  28.34us (  9.45us)         3
    nlp_grad  | 102.00us (102.00us) 102.06us (102.06us)         1
  nlp_grad_f  | 224.00us ( 74.67us) 223.71us ( 74.57us)         3
  nlp_hess_l  | 811.00us (405.50us) 811.83us (405.91us)         2
   nlp_jac_g  |  42.00us ( 14.00us)  41.33us ( 13.78us)         3
       total  |   2.68ms (  2.68ms)   2.67ms (  2.67ms)         1
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  29.00us ( 29.00us)  28.51us ( 28.51us)         1
       nlp_g  |   7.00us (  7.00us)   7.31us (  7.31us)         1
    nlp_grad  | 104.00us (104.00us) 104.22us (104.22us)         1
  nlp_grad_f  |  78.00us ( 78.00us)  78.26us ( 78.26us)         1
   nlp_jac_g  |  14.00us ( 14.00us)  13.90us ( 

----------- MPC execution -----------
Predicted position of cube [0.499999682618, 0.31677327478954, 0.2724911803854]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  89.00us ( 29.67us)  89.80us ( 29.93us)         3
       nlp_g  |  29.00us (  9.67us)  28.73us (  9.58us)         3
    nlp_grad  | 104.00us (104.00us) 104.61us (104.61us)         1
  nlp_grad_f  | 228.00us ( 76.00us) 227.17us ( 75.72us)         3
  nlp_hess_l  |   8.82ms (  4.41ms) 819.78us (409.89us)         2
   nlp_jac_g  |  46.00us ( 15.33us)  46.04us ( 15.35us)         3
       total  |  10.76ms ( 10.76ms)   2.75ms (  2.75ms)         1
----------- MPC execution -----------
Predicted position of cube [0.4999996762801, 0.32652383191909, 0.27249040567551]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  87.00us ( 29.00us)  86.82us ( 28.94us)         3
       nlp_g  |  28.00us (  9.33us)  27.82us (  9.27us)         3
    nlp_grad  | 108.00us (108.00us) 10

----------- MPC execution -----------
Predicted position of cube [0.49999809082555, 0.43299045436011, 0.084989297598358]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  98.00us ( 32.67us)  97.45us ( 32.48us)         3
       nlp_g  |  31.00us ( 10.33us)  30.00us ( 10.00us)         3
    nlp_grad  | 958.00us (958.00us) 113.19us (113.19us)         1
  nlp_grad_f  |   6.35ms (  2.12ms) 244.97us ( 81.66us)         3
  nlp_hess_l  |   3.69ms (  1.84ms) 832.50us (416.25us)         2
   nlp_jac_g  |   3.21ms (  1.07ms)  44.30us ( 14.77us)         3
       total  |  22.35ms ( 22.35ms)   2.93ms (  2.93ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999802642333, 0.43463954923507, 0.084989925087753]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  88.00us ( 29.33us)  88.08us ( 29.36us)         3
       nlp_g  |  28.00us (  9.33us)  27.70us (  9.23us)         3
    nlp_grad  | 100.00us (100.00

----------- MPC execution -----------
Predicted position of cube [0.49999413935664, 0.43632680985806, 0.084989999529349]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   1.03ms (344.67us) 107.58us ( 35.86us)         3
       nlp_g  |  98.00us ( 32.67us)  36.19us ( 12.06us)         3
    nlp_grad  |  97.00us ( 97.00us)  96.89us ( 96.89us)         1
  nlp_grad_f  |   2.12ms (708.00us) 258.87us ( 86.29us)         3
  nlp_hess_l  |  13.00ms (  6.50ms) 883.21us (441.60us)         2
   nlp_jac_g  | 173.00us ( 57.67us)  40.41us ( 13.47us)         3
       total  |  39.10ms ( 39.10ms)   3.18ms (  3.18ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999413959337, 0.43632680989699, 0.084989999826958]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  89.00us ( 29.67us)  89.74us ( 29.91us)         3
       nlp_g  |  26.00us (  8.67us)  26.69us (  8.90us)         3
    nlp_grad  | 105.00us (105.00

----------- MPC execution -----------
Predicted position of cube [0.49999413973103, 0.43632680991963, 0.084989999999997]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 101.00us ( 33.67us)  99.63us ( 33.21us)         3
       nlp_g  |  30.00us ( 10.00us)  29.57us (  9.86us)         3
    nlp_grad  |   1.21ms (  1.21ms) 136.74us (136.74us)         1
  nlp_grad_f  | 246.00us ( 82.00us) 246.58us ( 82.19us)         3
  nlp_hess_l  | 815.00us (407.50us) 816.21us (408.10us)         2
   nlp_jac_g  |  43.00us ( 14.33us)  43.37us ( 14.46us)         3
       total  |  14.06ms ( 14.06ms)   2.83ms (  2.83ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999413973103, 0.43632680991963, 0.084989999999997]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  92.00us ( 30.67us)  92.03us ( 30.68us)         3
       nlp_g  |  29.00us (  9.67us)  28.37us (  9.46us)         3
    nlp_grad  |  97.00us ( 97.00

...

----------- MPC execution -----------
Predicted position of cube [0.49999413973103, 0.43632680991963, 0.084989999999997]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  88.00us ( 29.33us)  87.15us ( 29.05us)         3
       nlp_g  |  28.00us (  9.33us)  27.50us (  9.17us)         3
    nlp_grad  | 101.00us (101.00us) 101.02us (101.02us)         1
  nlp_grad_f  |   1.82ms (608.00us) 226.95us ( 75.65us)         3
  nlp_hess_l  |  10.03ms (  5.02ms) 835.09us (417.54us)         2
   nlp_jac_g  |  67.00us ( 22.33us)  29.17us (  9.72us)         3
       total  |  17.46ms ( 17.46ms)   2.72ms (  2.72ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999413973103, 0.43632680991963, 0.084989999999997]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 663.00us (221.00us) 103.30us ( 34.43us)         3
       nlp_g  | 292.00us ( 97.33us)  31.36us ( 10.45us)         3
    nlp_grad  |   7.08ms (  7.08

----------- MPC execution -----------
Predicted position of cube [0.49999413973103, 0.43632680991963, 0.084989999999997]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  95.00us ( 31.67us)  94.88us ( 31.63us)         3
       nlp_g  |  29.00us (  9.67us)  29.01us (  9.67us)         3
    nlp_grad  | 110.00us (110.00us) 110.11us (110.11us)         1
  nlp_grad_f  | 237.00us ( 79.00us) 238.16us ( 79.39us)         3
  nlp_hess_l  |   8.81ms (  4.41ms) 808.26us (404.13us)         2
   nlp_jac_g  |  43.00us ( 14.33us)  43.63us ( 14.54us)         3
       total  |  11.03ms ( 11.03ms)   3.02ms (  3.02ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999413973103, 0.43632680991963, 0.084989999999997]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   3.89ms (  1.30ms)  95.59us ( 31.86us)         3
       nlp_g  |  29.00us (  9.67us)  28.87us (  9.62us)         3
    nlp_grad  | 114.00us (114.00

----------- MPC execution -----------
Predicted position of cube [0.49999413973103, 0.43632680991963, 0.084989999999997]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  97.00us ( 32.33us)  96.57us ( 32.19us)         3
       nlp_g  |  30.00us ( 10.00us)  29.01us (  9.67us)         3
    nlp_grad  | 113.00us (113.00us) 113.31us (113.31us)         1
  nlp_grad_f  | 235.00us ( 78.33us) 236.80us ( 78.93us)         3
  nlp_hess_l  | 803.00us (401.50us) 803.63us (401.81us)         2
   nlp_jac_g  |  43.00us ( 14.33us)  43.56us ( 14.52us)         3
       total  |   9.24ms (  9.24ms)   3.07ms (  3.07ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999413973103, 0.43632680991963, 0.084989999999997]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  85.00us ( 28.33us)  84.93us ( 28.31us)         3
       nlp_g  |  29.00us (  9.67us)  28.19us (  9.40us)         3
    nlp_grad  | 104.00us (104.00

----------- MPC execution -----------
Predicted position of cube [0.49999413973103, 0.43632680991963, 0.084989999999997]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  85.00us ( 28.33us)  84.37us ( 28.12us)         3
       nlp_g  |  27.00us (  9.00us)  28.02us (  9.34us)         3
    nlp_grad  |  98.00us ( 98.00us)  98.31us ( 98.31us)         1
  nlp_grad_f  | 223.00us ( 74.33us) 224.57us ( 74.86us)         3
  nlp_hess_l  |   8.82ms (  4.41ms) 818.77us (409.38us)         2
   nlp_jac_g  |  30.00us ( 10.00us)  30.02us ( 10.01us)         3
       total  |  10.70ms ( 10.70ms)   2.70ms (  2.70ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999413973103, 0.43632680991963, 0.084989999999997]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  91.00us ( 30.33us)  89.89us ( 29.96us)         3
       nlp_g  |  27.00us (  9.00us)  27.22us (  9.07us)         3
    nlp_grad  | 684.00us (684.00

----------- MPC execution -----------
Predicted position of cube [0.49999413973103, 0.43632680991963, 0.084989999999997]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  88.00us ( 29.33us)  87.00us ( 29.00us)         3
       nlp_g  |  28.00us (  9.33us)  27.88us (  9.29us)         3
    nlp_grad  |  97.00us ( 97.00us)  96.33us ( 96.33us)         1
  nlp_grad_f  | 228.00us ( 76.00us) 228.93us ( 76.31us)         3
  nlp_hess_l  |   1.45ms (726.50us) 810.11us (405.05us)         2
   nlp_jac_g  |  37.00us ( 12.33us)  35.83us ( 11.94us)         3
       total  |   6.17ms (  6.17ms)   2.64ms (  2.64ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999413973103, 0.43632680991963, 0.084989999999997]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  86.00us ( 28.67us)  87.23us ( 29.08us)         3
       nlp_g  |  29.00us (  9.67us)  28.44us (  9.48us)         3
    nlp_grad  | 113.00us (113.00

In [10]:
obj.end_simulation()

max(MPC_component.solver_times)

Ending simulation


0.016830921173095703